# CSPB 3155: Assignment 9 (45 points)

Topics: 
- Basics of Objects
- Type Constraints.


__Name__: Taylor Larrechea

In [36]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

cell36.sc:258: procedure syntax is deprecated: instead, add `: Unit =` to explicitly declare `passed`'s return type [quickfixable]
def passed(points: Int) {
                        ^


defined function passed

## Problem 1 (20 points): Objects in Scala

### A (12 points)
Consider the class below for an employee record in an organization that has three fields `name`, `eid` (employee id) and `salary`.

1) As given, scala will complain that class employee cannot extend the abstract class People. Add the missing methods to enable the class Employee to properly extend People

2) Write a factory pattern that takes in a formatted string of the form "John Doe,29585,33110" consisting of a string and two numbers, to extract the name and salaries for each employee. You may want to use `scala.util.matching.Regex` [https://www.scala-lang.org/api/2.9.2/scala/util/matching/Regex.html]
If the input fails to match the pattern, an `IllegalArgumentException` must be thrown

Companion objects are covered in scala book Chapter 4.3. Also lookup here: https://alvinalexander.com/scala/factory-pattern-in-scala-design-patterns

In [37]:
import scala.util.matching.Regex

abstract class People {
    /* I need the ability to convert every person to a string */
    def toString: String
    /* I need the ability to check if two people are the same */
    def equals (p: People): Boolean
}

/* 
  Note that for an employee to be equal to an object of type People, that object
  must be an employee with matching eid number 
*/

class Employee( val name: String, val eid: Int, var salary: Int) extends People {
    // YOUR CODE HERE
    override def toString: String = s"Employee(Name: $name, EID: $eid, Salary: $salary)"
    override def equals(p: People): Boolean = {
        p match {
            case e: Employee => e.eid == eid
            case _ => false
        }
    }
}

object Employee {
    def apply(formattedInput: String): Employee = {
        /* 
        Input format must be 
        (full name with possible spaces)[optional whitespaces],[optional whitespaces](employee ID number)[optional whitespaces],[optional whitespaces](employee salary)
        */
        // YOUR CODE HERE
        val pattern: Regex = """\s*(.+?)\s*,\s*(\d+)\s*,\s*(\d+)\s*""".r
        formattedInput match {
            case pattern(name, eid, salary) => {
                new Employee(name, eid.toInt, salary.toInt)
            }
            case _ => {
                throw new IllegalArgumentException("Invalid input format")
            }
        }
    }
}

import scala.util.matching.Regex
defined class People
defined class Employee
defined object Employee

In [38]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
println(e1.toString)
assert(e1.name == "Jonathan Doe", "Failed")
assert(e1.eid == 12893, "Failed")
assert(e1.salary == 39813, "Failed")
assert(e1.equals(new Employee("whoknows", 12893, 39813)))
passed(4)
//END TEST

Employee(Name: Jonathan Doe, EID: 12893, Salary: 39813)

*** Tests Passed (4 points) ***


e1: Employee = Employee(Name: Jonathan Doe, EID: 12893, Salary: 39813)

In [39]:
//BEGIN TEST
try {
    val e2 = Employee("Badly Formatted String, 29812, $3102")
    assert(false, "Failed, the string is badly formatted but your code is OK with it.")
} catch {
    case e => println(s"Expected behavior seen")
}
passed(4)
//END TEST

cell39.sc:6: This catches all Throwables. If this is really intended, use `case e : Throwable` to clear this warning.
    case e => println(s"Expected behavior seen")
         ^


Expected behavior seen

*** Tests Passed (4 points) ***


In [40]:
//BEGIN TEST
val e1 = Employee("Janet Maria Catherine Doe    , 12894   , 121331")
println(e1.toString)
assert(e1.name == "Janet Maria Catherine Doe", "Failed")
assert(e1.eid == 12894, "Failed")
assert(e1.salary == 121331, "Failed")
assert(e1.equals(new Employee("random", 12894, 121332)))
passed(4)
//END TEST

Employee(Name: Janet Maria Catherine Doe, EID: 12894, Salary: 121331)

*** Tests Passed (4 points) ***


e1: Employee = Employee(Name: Janet Maria Catherine Doe, EID: 12894, Salary: 121331)

### B (8 points)
We wish to add a new class of people to our system. These are "Contractor" with the information associated should involve their name (string), contract ID (Integer) and contract supervisor (Employee).

Write a new class named `Contractor` whose constructor should take in the name of the person (String), their ID number (Integer) and Supervisor (Employee) and extend from People. 

A contractor object is equal to an object p of type People, only if p is actually an instance of contractor, and if their names are the same and they have the same contract ID number.

In [41]:
// YOUR CODE HERE
class Contractor(val name: String, val contractorID: Int, val supervisor: Employee) extends People {
    override def toString: String = s"Name: $name, Contractor ID: $contractorID, Supervisor: ${supervisor.name}"
    override def equals(p: People): Boolean  = p match {
        case c: Contractor => this.name == c.name && this.contractorID == c.contractorID
        case _ => false
    }
}

defined class Contractor

In [42]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
val c1 = new Contractor("John Contract Worker", 12093, e1)
assert( !c1.equals(e1), "Failed")
assert( c1.equals(c1), "Failed")
passed(4)
//END TEST


*** Tests Passed (4 points) ***


e1: Employee = Employee(Name: Jonathan Doe, EID: 12893, Salary: 39813)
c1: Contractor = Name: John Contract Worker, Contractor ID: 12093, Supervisor: Jonathan Doe

In [43]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
val e2 = Employee("Jane Doe, 12894, 43814")
val c1 = new Contractor("John Contract Worker", 12093, e2)
val c2 = new Contractor("John Contract Worker", 12093, e1)
assert( c1.equals(c2), "Failed")
passed(4)
//END TEST


*** Tests Passed (4 points) ***


e1: Employee = Employee(Name: Jonathan Doe, EID: 12893, Salary: 39813)
e2: Employee = Employee(Name: Jane Doe, EID: 12894, Salary: 43814)
c1: Contractor = Name: John Contract Worker, Contractor ID: 12093, Supervisor: Jane Doe
c2: Contractor = Name: John Contract Worker, Contractor ID: 12093, Supervisor: Jonathan Doe

## Problem 2 (25 points): Traits in Scala

For this problem, we have defined two traits: `NumberOfLegs` that helps us define how many legs a given animal has and `WarmBlooded` that applies to warm blooded animals.

In [44]:
abstract class Animal 

trait NumberOfLegs {
    val nLegs: Int
    def getNumberOfLegs: Int = nLegs
}

trait WarmBlooded extends Animal {
    val bodyTempMaintained: Double
    def getBodyTemp: Double = bodyTempMaintained 
}

defined class Animal
defined trait NumberOfLegs
defined trait WarmBlooded

### A (5 points)

Define a class `Human` that inherits from `Animal` and mixes in the traits `NumberOfLegs` with `nLegs = 2` and `WarmBlooded` with `bodyTempMaintained = 98`. The class `Human` must take in a parameter called `name` of type `String` and implement a `getName` method without any parameters.

In [45]:
// YOUR CODE HERE
class Human(val name: String) extends Animal with NumberOfLegs with WarmBlooded {
    override val nLegs: Int = 2
    override val bodyTempMaintained: Double = 98.0
    def getName: String = name
}

defined class Human

In [46]:
//BEGIN TEST
val t1 = new Human("Jane Smith")
assert(t1.getNumberOfLegs == 2, "Your human does not have two legs")
assert(t1.bodyTempMaintained == 98.0, "Your human does not maintain a body temp of 98")
assert(t1.getBodyTemp == 98.0, "Your human's getBodyTemp Function is not working")
assert(t1.getName == "Jane Smith", "Your human's name is not setting correctly")
passed(5)
//END TEST


*** Tests Passed (5 points) ***


t1: Human = ammonite.$sess.cell45$Helper$Human@41049b11

### B (5 points)
Define a class named `Table` that mixes in the trait `NumberOfLegs` with `nLegs = 4`. 

_Note:_ `Table` cannot mixin the trait `WarmBlooded` (make sure you understand why that is as a quick check of concepts).

In [47]:
// YOUR CODE HERE
class Table extends NumberOfLegs {
    override val nLegs: Int = 4
}

defined class Table

In [48]:
//BEGIN TEST
val tbl = new Table()
assert(tbl.getNumberOfLegs == 4, "A Table must have four legs")
passed(5)
//END TEST


*** Tests Passed (5 points) ***


tbl: Table = ammonite.$sess.cell47$Helper$Table@408801fb

### C (15 points)
Define a class called `Menagerie` with type parameter `T` such that 

- T must always be a derived type of Animal with the trait NumberOfLegs (In scala you can say `T <: A with B` to indicate that a type parameter T must derive from class A with trait B applied).

- `Menagerie` has a list of objects of type `T`.

- The user must be able to instantiate an object of type Menagerie without providing any class parameters: `new Menagerie[T]`

- The class Menagerie has the method `addAnimalToMenagerie` that adds an object of type `T` to menagerie and returns a new Menagerie.

- Implement a method `addMultipleAnimalsToMenagerie` that adds more than one animal. 

- Add a method `totalLegs: Int` that sums up how many legs the animals in the menagerie have in total.

Here is the template for your solution below.

~~~
class Menagerie[T ..constraints..] ( ..class params..) {
    // ... Extra Stuff that you may want to add ...
   def addAnimalToMenagerie (a: T) : Menagerie[T] = {
    // Implement
   }
   def addMultipleAnimals(animals: List[T]): Menagerie[T] = {
    // Implement
   }
   def totalLegs:Int = {
    // Implement
   }
}
~~~


In [49]:
// YOUR CODE HERE
class Menagerie[T <: Animal with NumberOfLegs](animals: List[T] = List()) {
  private val animalList: List[T] = animals

  def addAnimalToMenagerie(a: T): Menagerie[T] = {
    new Menagerie(animalList :+ a)
  }

  def addMultipleAnimalsToMenagerie(animals: List[T]): Menagerie[T] = {
    new Menagerie(animalList ++ animals)
  }

  def totalLegs: Int = animalList.map(_.getNumberOfLegs).sum
}

defined class Menagerie

In [50]:
//BEGIN TEST SETUP
class Donkey extends Animal with NumberOfLegs with WarmBlooded {
    val nLegs:Int = 4
    val bodyTempMaintained = 96.0
}

class Lion extends Animal with NumberOfLegs {
    val nLegs:Int = 4
}

class Mule extends Donkey

val d1 = new Donkey()
val d2 = new Donkey()
val d3 = new Donkey()
val d4 = new Mule()
//END TEST SETUP

defined class Donkey
defined class Lion
defined class Mule
d1: Donkey = ammonite.$sess.cell50$Helper$Donkey@64c3d187
d2: Donkey = ammonite.$sess.cell50$Helper$Donkey@118b154d
d3: Donkey = ammonite.$sess.cell50$Helper$Donkey@425d7bcd
d4: Mule = ammonite.$sess.cell50$Helper$Mule@4a8c5661

In [51]:
//BEGIN TEST
val m1 = new Menagerie[Donkey](Nil)
passed(3)
//END TEST


*** Tests Passed (3 points) ***


m1: Menagerie[Donkey] = ammonite.$sess.cell49$Helper$Menagerie@12f3a7ed

In [52]:
//BEGIN TEST
val m1 = new Menagerie[Donkey](Nil)
val m2 = m1.addAnimalToMenagerie(d1)
val n = m2.totalLegs
assert(n == 4, s"1 quadraped = 4 legs. You have $n legs : are you missing legs? Too many legs?")
passed(6)
//END TEST


*** Tests Passed (6 points) ***


m1: Menagerie[Donkey] = ammonite.$sess.cell49$Helper$Menagerie@23cf8d44
m2: Menagerie[Donkey] = ammonite.$sess.cell49$Helper$Menagerie@722c94d4
n: Int = 4

In [53]:
//BEGIN TEST
val m1 = new Menagerie[Donkey](Nil)
val m2 = m1.addMultipleAnimalsToMenagerie(List(d1, d2, d3, d4))
val n = m2.totalLegs
assert(n == 16, s"4 quadrapeds = 16 legs. You have $n legs : are you missing legs? Too many legs?")
passed(6)
//END TEST


*** Tests Passed (6 points) ***


m1: Menagerie[Donkey] = ammonite.$sess.cell49$Helper$Menagerie@32ad8323
m2: Menagerie[Donkey] = ammonite.$sess.cell49$Helper$Menagerie@60cf93db
n: Int = 16